In [1]:
pip install torch transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

In [5]:
# Пример данных
texts = ["Отличный товар, быстрая доставка",
         "Не соит покупать, сломался при первом запуске",
         "Мне не понравилось. Выглядит дешево и непрезентабельно",
         "Упаковано хорошо, в деле наче не пробовал",
         "Отличный продукт! Пользуюсь уже месяц, и результат превзошел все ожидания.",
         "Очень доволен покупкой! Качество на высоте, рекомендую всем.",
         "Не работает так, как было заявлено. Не рекомендую.",
         "Неудобный и непрактичный. Не стоит своих денег.",
         "Просто супер! Пользуюсь с удовольствием и не жалею о покупке.",
         "Доставка задержалась, а сам товар оказался бракованным."
         ]
labels = [1, 0, 0, 1, 1, 1, 0, 0, 1, 0]  # 1 - положительный, 0 - отрицательный

In [6]:
# Разделение данных на обучающую и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.3, stratify=labels)

In [7]:
# Загрузка токенизатора RuBERT
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [8]:
# Создание пользовательского датасета
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [9]:
# Создание датасетов и загрузчиков данных
train_dataset = TextDataset(train_texts, train_labels)
test_dataset = TextDataset(test_texts, test_labels)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)

In [10]:
# Загрузка модели RuBERT для классификации
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Оптимизатор
optimizer = AdamW(model.parameters(), lr=1e-5)

In [12]:
# Обучение модели
model.train()
for epoch in range(5):  # Количество эпох небольшое для демонстрации
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.7024757862091064
Epoch 0, Loss: 0.696247398853302
Epoch 0, Loss: 0.6777313947677612
Epoch 0, Loss: 0.9239165186882019
Epoch 1, Loss: 0.6821572780609131
Epoch 1, Loss: 0.6757364869117737
Epoch 1, Loss: 0.5696672201156616
Epoch 1, Loss: 0.5571925640106201
Epoch 2, Loss: 0.7062034010887146
Epoch 2, Loss: 0.6826804876327515
Epoch 2, Loss: 0.6547814607620239
Epoch 2, Loss: 0.5579032897949219
Epoch 3, Loss: 0.600510835647583
Epoch 3, Loss: 0.5503012537956238
Epoch 3, Loss: 0.6267344951629639
Epoch 3, Loss: 0.6335680484771729
Epoch 4, Loss: 0.6226837635040283
Epoch 4, Loss: 0.4836690127849579
Epoch 4, Loss: 0.46022436022758484
Epoch 4, Loss: 0.6816543340682983


In [13]:
# Оценка модели
model.eval()
correct_predictions = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

accuracy = correct_predictions.double() / len(test_dataset)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6667


Качество сильно зависиот от того какие экземпляры попали в тестовую выборку нашего небольшого датасета, а также от того, что модель обучали на небольшом количестве эпох

In [14]:
# Функция для предсказания
def predict(texts):
    model.eval()  # Устанавливаем модель в режим оценки
    predictions = []

    with torch.no_grad():
        for text in texts:
            # Токенизация текста
            encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
            input_ids = encoding['input_ids']
            attention_mask = encoding['attention_mask']

            # Получение предсказаний
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.append(preds.item())  # Сохраняем предсказание

    return predictions

# Примеры текстов для предсказания
new_texts = [
    "Прекрасный сервис и отличное качество. Буду заказывать снова!!!",
    "Я очень не доволен своей покупкой.",
    "Качество материалов плохое. Не рекомендую.",
    "Быстрая доставка, отличный сервис, но сломался через неделю, по гарантии не оформляют",
]

# Получение предсказаний
predictions = predict(new_texts)

# Вывод результатов
for text, pred in zip(new_texts, predictions):
    sentiment = "Положительный" if pred == 1 else "Отрицательный"
    print(f"Текст: \"{text}\" - Предсказание: {sentiment}")


Текст: "Прекрасный сервис и отличное качество. Буду заказывать снова!!!" - Предсказание: Положительный
Текст: "Я очень не доволен своей покупкой." - Предсказание: Положительный
Текст: "Качество материалов плохое. Не рекомендую." - Предсказание: Положительный
Текст: "Быстрая доставка, отличный сервис, но сломался через неделю, по гарантии не оформляют" - Предсказание: Отрицательный


Обратите внимание, последний отзыв противоречивый

Дополнительное задание:

Используя небольшой набор данных оп отзывам на продукты, сотсоящий из трех классов:

Положительный: https://huggingface.co/datasets/ankislyakov/feedback/resolve/main/positive.txt
Нейтральный: https://huggingface.co/datasets/ankislyakov/feedback/resolve/main/neutral.txt
Негативный: https://huggingface.co/datasets/ankislyakov/feedback/resolve/main/negative.txt

Выполнить трансфертное обучение модели RuBert для классификации отзывов